##### Author contributions
Please fill out for each of the following parts who contributed to what:
- Conceived ideas: 
- Performed math exercises: 
- Performed programming exercises:
- Contributed to the overall final assignment: 

# Chapter 9
## Boltzmann machines

    Hand-in bug-free (try "Kernel" > "Restart & Run All") and including all (textual as well as figural) output via Brightspace before the deadline (see Brightspace).
    
Learning goals:
1. Implement a Boltzmann machine

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import urllib
import skimage as ski

A (unrestricted) Boltzmann machine is like a Hopfield network with stochastic state updates. The degree of randomness allows for investiations both in Hopfield networks and in how real associative memory may work (biological neurons are stochastic, too; and there are ideas that they make use of the irreducible noise). 

### Exercise 1: Flip-flop  (2 points)

This is the **Boltzmann distribution** for a state $\mathbf{x}$:

$p(\mathbf{x}) = \frac{1}{Z} \exp( \frac{-E(\mathbf{x})}{T} ) $

with the state sum $Z$: 

$Z = \sum_{\mathbf{x}}  \exp(\frac{-E(\mathbf{x})}{T})$


Use it to derive the stable probability distribution of the **flip-flop** (2 units with bias $0.5$, connected with the identical weight $w_{12} = w_{21} = −1$). 

Assume the temperature $T=1$. Start by first computing $E(\mathbf{x})$ for each possible state of the Hopfield network.

### Solution 1
$\LaTeX$ here.

Now you will implement your own Boltzmann machine. As in the Hopfield network assignment, we will use an image and its mirrored version as the patterns the Boltzmann machine should learn. 

Note that we use the binary representation here, not the bipolar one. 

In [ ]:
# The source image
f = urllib.request.urlopen("https://homepages.cae.wisc.edu/~ece533/images/watch.png")

# Read the image
x1 = mpimg.imread(f)
x1 = ski.transform.resize(x1, (76, 102), mode="reflect", anti_aliasing=True)

# Make 2D and binary
x1 = np.mean(x1, axis=2)
x1[x1 < np.mean(x1.flatten())] =  0 # Black
x1[x1 >= np.mean(x1.flatten())] = 1 # White
x1.astype("int32")

# Make duplicate but mirrored second image
x2 = np.fliplr(x1)

# Flatten images
sz = x1.shape
X = np.stack((x1.flatten(), x2.flatten()), axis=1)

In [ ]:
# Plot the images
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(X[:, 0].reshape(sz), cmap="gray")
ax[0].set_xticks([], [])
ax[0].set_yticks([], [])
ax[0].set_title("x1")

ax[1].imshow(X[:, 1].reshape(sz), cmap="gray")
ax[1].set_xticks([], [])
ax[1].set_yticks([], [])
ax[1].set_title("x2")

### Exercise 2: The sigmoid (1 point)
Write a function `sigmoid(x)` that computes the *sigmoid activation function* $\sigma(x)=\frac{1}{1+exp(-x)}$.

### Solution 2

In [ ]:
# The activation function
## Code here ##

### Exercise 3: Probabilities (1 point)

Write a function `compute_probability(w, x, b, T)` that returns the probability that the state of node $x_i$ is set to $1$ ($0$ instead): 

$P(x_i=1)=\sigma(\frac{1}{T}(\mathbf{w}_i^{\top}\mathbf{x}+b_i))$

where $T$ is the temperature, $\mathbf{w}_i$ the weights and $b_i$ the bias of node $i$; and $\mathbf{x}$ the current state vector. $\sigma$ is the sigmoid activation function.

### Solution 3

In [ ]:
# Compute probabilities
## Code here ##

### Exercise 4: Gibbs sampling  (2 points)
Write a function `gibbs_sampling(w, b, temperature=1.0, n_gibbs=20, n_burnin=10)` that approximates the model distribution for training a Boltzmann machine via Gibbs sampling. The Boltzmann machine is given by the current weights `w` and biases `b`.

1. Create an array `X` for saving the node states for each time step. Then initialize a random initial node state vector $\mathbf{x}^{(1)}$ and save it in `X[:,0]`. These are the node states for $t=1$.
1. For $t=2, ..., N$, where $N$ is the number of Gibbs sampling steps `n_gibbs`:
    1. Compute the probability $P(x^{t+1}_i = 1)$ for all node states $x^{t}_i$ in `X[:,t]` (the previous state vector). 
    1. Determine the new binary state $x^{t+1}_i$ by drawing a random number between 0 and 1 using `np.random.rand()`, and comparing it with $P(x^{t+1}_i = 1)$. That is, depending on the probability for $1$, it should be set to $1$ or $0$ at time `t+1`. Save the new state in `X[i,t+1]`. 
1. Return `X` without the burn-in phase, that is without the first `n_burnin` samples. 

### Solution 4

In [ ]:
def gibbs_sampling(w, b, temperature=1.0, n_gibbs=20, n_burnin=10):
    n_nodes = w.shape[0]
    
    # Initialize states
    X = np.zeros((n_nodes, n_gibbs))
                 
    # Initialize first state vector
    ## Code here ##
                 
    # Loop over Gibbs samples
    ## Code here ##
                 
        # Loop over nodes
        ## Code here ##
                 
            # Compute probability for state 1
            ## Code here ##
                 
            # Sample whether it should change to 1
            ## Code here ##

    # Discard burn-in
    ## Code here ##
    
    return X

### Exercise 5: Expectations  (1 point)
Write a function `compute_expectations(X)` that computes the expectation (~mean over patterns / samples) of the partial derivatives for the weights and bias terms, given as: 

$$\frac{\partial J(x)}{\partial w_{ij}}=-x_ix_j$$
$$\frac{\partial J(x)}{\partial b_i}=-x_i$$

When running training you will compute these under the empirical and the model distribution (given as input `X`, which contains multiple patterns / sampling steps respectively). Note that if you use the dot product to compute $\frac{\partial J(x)}{\partial w_{ij}}$ you need to divide by the number of patterns / samples `N` in `X`. Check for yourself (on paper, with a small example matrix `X` with 2 patterns) why this is so when you make use of the dot product to compute all $- x_i x_j$ at once. It will also help you arranging the dot product correctly. 

Hint: $I$ is the number of nodes. Then the partial derivative for the weight update `dw` should have the shape $(I, I)$. 
You also need to make sure that the diagonal is set so that the Boltzmann machine converges. You can complete this exercise in 3 lines of code.  

### Solution 5

In [ ]:
# Compute expectations
## Code here ##

### Exercise 6: Boltzmann training  (1 point)
Now, with the components you wrote above, you can fill in the missing part in `boltzmann_train`. What is missing is updating the weights `w` and biases `b` with the information gathered from the empirical and model distribution, and the learning rate `eta`. Add the two lines performing this step below. 

### Solution 6

In [ ]:
def boltzmann_train(XE, temperature=1.0, eta=0.01, n_epochs=5, n_gibbs=20, n_burnin=10):
    n_nodes, n_examples = XE.shape

    # Initialize weights:
    w = np.zeros((n_nodes, n_nodes))
    
    # Initialize biases:
    b = np.zeros(n_nodes)
    
    # Compute expectations under the empirical distribution. As we compute this on 
    # the training patterns, we only need to do it once: 
    dE_dw, dE_db = compute_expectations(XE)     

    # Loop over epochs: 
    for i_epoch in range(n_epochs):
        print("Epoch {}/{}.".format(1 + i_epoch, n_epochs))
        
        # Gibbs sampling with the current model:
        XM = gibbs_sampling(w, b, temperature, n_gibbs, n_burnin)

        # Compute expectations under the model distribution: 
        dEM_dw, dEM_db = compute_expectations(XM)

        # Update weights and biases
        ## Code here ##
    
    # Force symmetry
    w = (w + w.T) / 2
    
    print("Training done.")
    
    return w, b

### Exercise 7: Dreaming (1 point)

Write a function `boltzmann_dream(w, b, temperature=1.0, n_epochs=20)` that you can use to sample images from a previously trained Boltzmann machine (given as `w` and `b`) that is left running on its own for `n_epochs`. This boils down to performing Gibbs sampling starting with a random state vector. Use your previously implemented functions. 

### Solution 7

In [ ]:
# Boltzmann dreaming
## Code here ##

### Exercise 8 (1 point)

**1.** Train your Boltzmann machine with `boltzmann_train` on the two images saved in `X` (the empirical distribution `XE`). Use $T=1$, 20 Gibbs sampling steps, a burn-in phase of 10, 3 training epochs, and a learning rate of $0.01$. Then let your trained Boltzmann machine dream for 20 epochs via `boltzmann_dream`. Plot the last 5 states as images. 

**2.** Now, change the temperature to $T=5$. Again, train the Boltzmann machine, let it run freely for 20 epochs and then plot the last 5 states it creates. 

### Solution 8

In [ ]:
# Train Boltzmann machine
## Code here ##

# Test Boltzmann machine
## Code here ##

# Plot last 5 samples
## Code here ##


In [ ]:
# Train Boltzmann machine
## Code here ##

# Test Boltzmann machine
## Code here ##

# Plot last 5 samples
## Code here ##
